In [2]:
import pandas as pd

In [3]:
# Read ismr file 
data = pd.read_csv("ljic219b15.20_.ismr", header=None, usecols=[0,1,2,7,4,5], squeeze=True)
#
WN = data[0] # Week number, since first epoch 
TOW = data[1] # Time of week (s)
PRN = data[2]
S4 = data[7] # S4 in "Sig1"
alpha = data[4]
epsilon = data[5]

In [4]:
# GPS time: week & seconds; to UTC time.
# OJO: It's missing the leapseconds, these value get from the navigation file
def WeekSeconds2UTC(gpsweek,gpsseconds,leapseconds):
    import datetime
    
    datetimeformat = "%Y-%m-%d %H:%M:%S"
    first_epoch = datetime.datetime.strptime("1980-01-06 00:00:00",datetimeformat)
    elapsed = datetime.timedelta(days=(gpsweek*7),seconds=(gpsseconds-leapseconds))
    
    # LISN date format: 2 digit year, day of year, seconds since midnight for a day 
    date = datetime.datetime.strftime(first_epoch + elapsed,"%y-%j;%H:%M:%S")
    
    year = int(date[:2])
    day = int(date[3:6])
    hour = int(date[7:9])
    minute = int(date[10:12])
    second = int(date[13:15])
    seconds_day = second*1 + minute*60 + hour*60*60
    
    return {"year":year, "day_year":day, "seconds_day": seconds_day}
    

In [6]:
# The PRNs code were obtained from PolaRx5S Reference Guide
def get_PRN(svid):
    if 1<=svid<=37:
        prn = "G"+str(svid)
    elif 38<=svid<=61:
        prn = "R"+str(svid-37)
    elif svid==62:
        prn = "NA"
    elif 63<=svid<=68:
        prn = "R"+str(svid-38)
    elif 71<=svid<=106:
        prn = "E"+str(svid-70)
    elif 107<=svid<=119:
        prn = "NA"
    elif 120<=svid<=140:
        prn = "S"+str(svid-100)
    elif 141<=svid<=177:
        prn = "C"+str(svid-140)
    elif 181<=svid<=187:
        prn = "J"+str(svid-180)
    elif 191<=svid<=197:
        prn = "I"+str(svid-190)
    elif 198<=svid<=215:
        prn = "S"+str(svid-157)
    elif 216<=svid<=222:
        prn = "I"+str(svid-208)
    else:
        prn = "svid not valid!"
    
    return prn

In [7]:
# Create new columns of lisn file 
yr = [] # year
dyr = [] # day_year
sd = [] # seconds_day
nd = [1]*len(WN)
prn = [] # prn
#
for i in range(len(WN)):
    fecha = WeekSeconds2UTC(int(WN[i]), int(TOW[i]), 0)
    yr.append(fecha["year"])
    dyr.append(fecha["day_year"])
    sd.append(fecha["seconds_day"])   
    prn.append(get_PRN(int(PRN[i])))

In [10]:
# Create the lisn dataframe
list_tuples = list(zip(yr,dyr,sd,nd,prn)) # creates a list of tuples
dataFrame = pd.DataFrame(list_tuples, columns=['year','day','seconds','Ndata','PRN']) # creates a pandas dataframe

# Adding other pandas series
dataFrame['S4'] = S4
dataFrame['alpha'] = alpha
dataFrame['epsilon'] = epsilon

dataFrame

,year,day,seconds,Ndata,PRN,S4,alpha,epsilon
0,20,219,4560,1,G10,0.097,3.0,26.0
1,20,219,4560,1,G12,0.147,61.0,14.0
2,20,219,4560,1,G18,0.043,169.0,64.0
3,20,219,4560,1,G20,0.040,38.0,49.0
4,20,219,4560,1,G21,0.045,304.0,36.0
...,...,...,...,...,...,...,...,...
492,20,219,5400,1,C22,0.105,46.0,28.0
493,20,219,5400,1,C24,0.107,223.0,27.0
494,20,219,5400,1,C26,0.025,269.0,71.0
495,20,219,5400,1,C34,0.163,274.0,11.0


In [12]:
# Select a subset of data 
subset1 = dataFrame[dataFrame["seconds"] == 4560]
subset1.head(3)

,year,day,seconds,Ndata,PRN,S4,alpha,epsilon
0,20,219,4560,1,G10,0.097,3.0,26.0
1,20,219,4560,1,G12,0.147,61.0,14.0
2,20,219,4560,1,G18,0.043,169.0,64.0


In [13]:
# Select the header
header = list(subset1.iloc[0,:3])
header.append(len(subset1))
header

[20, 219, 4560, 33]

In [20]:
# Select the body
ss1 = subset1.loc[:,"PRN":] # Select some columns
ss1

,PRN,S4,alpha,epsilon
0,G10,0.097,3.0,26.0
1,G12,0.147,61.0,14.0
2,G18,0.043,169.0,64.0
3,G20,0.040,38.0,49.0
4,G21,0.045,304.0,36.0
5,G25,0.051,78.0,43.0
6,G26,0.128,207.0,29.0
7,G29,0.121,141.0,20.0
8,G31,0.040,269.0,54.0
9,G32,0.173,333.0,10.0


In [15]:
# Create a list of values in the lisn format 
for i in range(len(ss1)):
    header += list(ss1.iloc[i])
header

[20,
 219,
 4560,
 33,
 'G10',
 0.09699999999999999,
 3.0,
 26.0,
 'G12',
 0.147,
 61.0,
 14.0,
 'G18',
 0.043,
 169.0,
 64.0,
 'G20',
 0.04,
 38.0,
 49.0,
 'G21',
 0.045,
 304.0,
 36.0,
 'G25',
 0.051,
 78.0,
 43.0,
 'G26',
 0.128,
 207.0,
 29.0,
 'G29',
 0.121,
 141.0,
 20.0,
 'G31',
 0.04,
 269.0,
 54.0,
 'G32',
 0.17300000000000001,
 333.0,
 10.0,
 'R5',
 0.196,
 56.0,
 10.0,
 'R9',
 0.035,
 197.0,
 77.0,
 'R10',
 0.111,
 205.0,
 35.0,
 'R16',
 0.085,
 28.0,
 34.0,
 'R20',
 0.12300000000000001,
 209.0,
 41.0,
 'R21',
 0.076,
 278.0,
 26.0,
 'E1',
 0.039,
 68.0,
 60.0,
 'E11',
 0.276,
 306.0,
 11.0,
 'E12',
 0.06,
 268.0,
 54.0,
 'E19',
 0.28600000000000003,
 31.0,
 11.0,
 'E21',
 0.174,
 44.0,
 12.0,
 'E31',
 0.046,
 195.0,
 56.0,
 'S25',
 0.124,
 83.0,
 20.0,
 'S31',
 0.051,
 284.0,
 42.0,
 'S33',
 0.084,
 279.0,
 29.0,
 'S38',
 0.037000000000000005,
 289.0,
 52.0,
 'C12',
 0.08199999999999999,
 262.0,
 27.0,
 'C19',
 0.092,
 93.0,
 15.0,
 'C22',
 0.109,
 40.0,
 25.0,
 'C24',
 0.0

In [17]:
# Create a row dataframe 
parameters = pd.DataFrame([header])
parameters

,0,1,2,3,4,5,6,7,8,9,...,126,127,128,129,130,131,132,133,134,135
0,20,219,4560,33,G10,0.097,3.0,26.0,G12,0.147,...,250.0,70.0,C34,0.11,280.0,9.0,C35,0.095,165.0,34.0


In [19]:
# Concatenate rows 
result = parameters.append(parameters, ignore_index=True)
result

,0,1,2,3,4,5,6,7,8,9,...,126,127,128,129,130,131,132,133,134,135
0,20,219,4560,33,G10,0.097,3.0,26.0,G12,0.147,...,250.0,70.0,C34,0.11,280.0,9.0,C35,0.095,165.0,34.0
1,20,219,4560,33,G10,0.097,3.0,26.0,G12,0.147,...,250.0,70.0,C34,0.11,280.0,9.0,C35,0.095,165.0,34.0
